In [1]:
%autoreload 2

In [38]:
from collections import defaultdict
import random

from IPython.core.display import display, HTML
from tqdm.notebook import tnrange, tqdm

from mutation_experiments import *
import ast_printer

In [3]:
args = parser.parse_args([])

grammar = open(args.grammar_file).read()
grammar_parser = tatsu.compile(grammar) 

asts = load_asts(args, grammar_parser)

In [4]:
NUM_GAMES = 100

regrwoth_mutated_asts = []

for seed in tnrange(NUM_GAMES):
    validators = [ASTVariableValidator(), ASTScoringPreferenceValidator()]
    sampler = ASTRegrowthSampler(grammar_parser, asts, mutation_prob=0.5, validators=validators)
    random.seed(seed)
    try:
        mutated_ast = sampler.sample_regrowth()
    except SamplingFailedException:
        continue
    regrwoth_mutated_asts.append((mutated_ast, seed))
        


In [5]:
NUM_GAMES = 100
START = 100
single_step_mutated_asts = []

for seed in tnrange(NUM_GAMES):
    validators = [ASTVariableValidator(), ASTScoringPreferenceValidator()]
    sampler = ASTRegrowthSampler(grammar_parser, asts, mutation_prob=0.5, validators=validators)
    random.seed(START + seed)
    try:
        mutated_ast = sampler.sample_single_step()
    except:
        continue
    single_step_mutated_asts.append((mutated_ast, START + seed))
        


In [64]:
def ast_to_buffer(ast, html=True):
    ast_printer.reset_buffers(True)
    context = dict()
    if html:
        context['html'] = True
    ast_printer.pretty_print_ast(ast, context=context)


def new_ast_to_buffer(new_asts, index, html=True):
    mutated_ast, _ = new_asts[index]
    ast_to_buffer(ast, html=html)


def visualize_ast(ast, html=True):
    ast_to_buffer(ast, html=html)
    out_str = '\r\n'.join(ast_printer.BUFFER)
    if html:
        display(HTML(out_str))
    else:
        print(out_str)


def visualize(new_asts, index, html=True):
    mutated_ast, seed = new_asts[index]
    print(f'With random seed {seed}, mutated:\r\n')
    visualize_ast(mutated_ast, html=html)
    


# Games generated with full regrwoth

I'll pring out some interesting games, and try to explain the game and the change that was made. 

The color-coding on these is perhaps not the most informative, but:
* <span style="color: #AA00FF;">Purple:</span> the original node substituted in
* <span style="color: #FF6A00;">Orange:</span> any nodes sampled under the original node from another game
* <span style="color: #00FF80;">Green:</span> nodes from the substituted subtree (the purple one) where we replaced at least one value under the node, but not the entire node.

**TL;DR: if it's colored, it was't in the original game**


In the game below, it changed the quantification and type from `?g - game_object` to `?g - dodgeball`.

Originally, that clause was marking keeping the beachball (`?b`) in the air for 20 counts -- by providing it's not or touching any other object.

Now it stipulates the beachball can't touch or be in the dodgeball, which is rather more trivial


In [7]:
visualize(regrwoth_mutated_asts, 0)

With random seed 0, mutated:



This is another similar change. Originally, this quantified `?b - bed` like in the previous game, to mark that the player should make the throw from the bed. 

With this change, it marks that the player should make the throw while standing on a block, which is cute.

In [8]:
visualize(regrwoth_mutated_asts, 2)

With random seed 2, mutated:



Funny failure mode -- regardless of what this attempted to convey originally, this can never occur, obviously.

In [9]:
visualize(regrwoth_mutated_asts, 4)

With random seed 4, mutated:



Another amusing failure mode -- ths mutation performed substituted in the predicate `(on ?1 ?2)`, and renaming variables to match the local context created an identical predicate to the one that's already there. 

In [10]:
visualize(regrwoth_mutated_asts, 9)

With random seed 9, mutated:



Changes the scoring from awarding five points for each succesful throw (`throwLandsInTarget`) to five points for each throw. Makes the game easier?

In [11]:
visualize(regrwoth_mutated_asts, 11)

With random seed 11, mutated:



Another amusing failure mode that occurs a bunch -- a predicate being called with the wrong number of arguments as result of a mutation.

Semantically, I considered `(agent_holds ?1)` to accept one argument, and `(on ?1 ?2)` to accept two, and here they ended up flipped.

In [12]:
visualize(regrwoth_mutated_asts, 15)

With random seed 15, mutated:



This game originally was something like "consturct a a tunnel from the flat and bridge blocks, then throw the ball under the bridge.

The change essentially makes it such that you have to construct a tunnel from the ramp and bridge blocks, but it doesn't enforce the first setup term, which was designed to make sure this bridge is constructed on the floor.

In [13]:
visualize(regrwoth_mutated_asts, 25)

With random seed 26, mutated:



Changes the terminal conditions, which were originally `chairStoppedRotating` happening once (that is, the chair stops rotating), to `objectLandsOnRotatingChair` happening once, that is, ending at the first succesful shot. 

Not nonsense, but doesn't make too much sense either.

In [14]:
visualize(regrwoth_mutated_asts, 27)

With random seed 28, mutated:



Changes this game from balancing on the large triangular ramp (what it was before) to balancing on a cube block, which sounds like it would be rather harder?

In [15]:
visualize(regrwoth_mutated_asts, 40)

With random seed 41, mutated:



Game changed from bowling a dodgeball to bowling a cube block

In [16]:
visualize(regrwoth_mutated_asts, 42)

With random seed 43, mutated:



Another minor semantic change, from bouncing a ball off the wall and catching it, to bouncing a ball off the curved wooden ramp and catching it.

In [17]:
visualize(regrwoth_mutated_asts, 53)

With random seed 55, mutated:



# Games generated by making a single step

In these the color scheme is simpler -- the mutation is in blue (first color in a equential colormap, assuming I might want to eventually mark each mutation, or something)

Changed the setup slightly -- before it was `(game-conserved (= (distance ?b ?r) 2))`, that is, the ramp is 2 meters from the bed, and now it's that the ramp is on the bed. This works.

In [18]:
visualize(single_step_mutated_asts, 3)

With random seed 103, mutated:



Another minor semantic difference -- from ending the game after three roll attempts, to ending it after three succesful rolls.

In [19]:
visualize(single_step_mutated_asts, 6)

With random seed 106, mutated:



A small change to the scoring -- if you manage to knock off the desk lamp at least once, you get an extra point for each time you manage to knock off the desktop.

In [20]:
visualize(single_step_mutated_asts, 7)

With random seed 107, mutated:



I think the other substitution method also did something like this -- though it bounced it off the curved ramp, and this bounces it off the hexagonal bin

In [21]:
visualize(single_step_mutated_asts, 19)

With random seed 121, mutated:



Throw the cellphone such that it is adjacent to the doggie bed, rather than such that it lands on it. Cute and valid.

In [22]:
visualize(single_step_mutated_asts, 21)

With random seed 123, mutated:



Originally, that preference asked to to the chair with the pillow. Now it asks that after throwing the pillow, and before the pillow stops moving (or the agent picks it up again), the chair will move. Presumably this could happen because the agent moves it, or the pillow hits it. Totally valid.

In [23]:
visualize(single_step_mutated_asts, 24)

With random seed 126, mutated:



I'm mildly stunned this worked, but it worked -- this is what was there before: `(exists (?b2 - (either cube_block tall_cylindrical_block short_cylindrical_block)) (on ?b1 ?b2))`


Now it asks if there's another game object, that is in the building, and `?b1` is on the game object `?o2` -- it basically changes the game from "build a tower with the blocks" to "build a tower, starting from a block, and using any object you want, but you only get points for each block you use" -- so you could use other objects to stabilize, for example.

In [24]:
visualize(single_step_mutated_asts, 38)

With random seed 145, mutated:



This change to the setup is mostly coherent (even if one of the terms is repeated, although with the arguments in the other order), but I don't know how much semantic sense it makes. If I were to try to describe it in natural language:

"Place two blocks next to the chair and to each other. One of those blocks should be on another block. Place yet another block next to the block you put on top, and place another block next to the block you just placed. Finally, place the last block on the floor, next to the block you placed other blocks on."

In [25]:
visualize(single_step_mutated_asts, 39)

With random seed 146, mutated:



Slightly changed how the preference is specified -- previously it was `(hold (and (not (agent_holds ?d)) (in_motion ?d))) `, that is, the agent doesn't touch the ball while it's in motion, and now it specifies that the block can't move while the ball is in motion, until they touch. 

In other words, previously you could move the block to make the moving ball touch it, and now you can make contact with the ball (while not stopping its motion) but not the block.

In [26]:
visualize(single_step_mutated_asts, 40)

With random seed 147, mutated:



Changed from a "throw after dribbling game" to simpler "throw in the bin" game.

In [27]:
visualize(single_step_mutated_asts, 42)

With random seed 150, mutated:



Originally that predicate was a much more complicated one to count an object landing on the chair, or on another object on the chair: 
```
(or 
    (on ?c ?o)
    (exists (?o2 - game_object) (and 
        (not (= ?o ?o2))
        (in_building ?o2)
        (on ?o2 ?o)
    ))
)
```
Now, it counts simpler occurences -- either the chair ending up on object, which is a little nonsensical, or an object touching the chair, which makes more sense.

In [28]:
visualize(single_step_mutated_asts, 56)

With random seed 168, mutated:



Changed from the agent dribbling around a cube block, to the agent dribbling around those items quantified.

In [29]:
visualize(single_step_mutated_asts, 62)

With random seed 174, mutated:



# Second mutation of interesting games

In [98]:
INDICES_TO_MUTATE = (3, 6, 7, 19, 21, 24, 38, 39, 40, 42, 56, 62,)
NUM_MUTATIONS_TO_GENERATE = 10
MAX_GENERATION_ATTEMPTS = 100

second_step_mutations_dict = defaultdict(list)

validators = [ASTVariableValidator(), ASTScoringPreferenceValidator()]
sampler = ASTRegrowthSampler(grammar_parser, asts, mutation_prob=0.5, validators=validators)

for idx in tqdm(INDICES_TO_MUTATE):
    src_ast = single_step_mutated_asts[idx][0]
    seed = -1
    while seed < MAX_GENERATION_ATTEMPTS and len(second_step_mutations_dict[idx]) < NUM_MUTATIONS_TO_GENERATE:
        seed += 1
        random.seed(seed)
        try:
            src_ast_copy = copy_ast(grammar_parser, src_ast)
            copy_mutation_tags(src_ast, src_ast_copy)
            mutated_ast = sampler.sample_single_step(src_ast_copy, n_mutations=1)
        except:
            continue
        second_step_mutations_dict[idx].append((mutated_ast, seed))
        




# Visualizing choice second mutations
The second mutation will be in orange

**Original note**: Changed the setup slightly -- before it was `(game-conserved (= (distance ?b ?r) 2))`, that is, the ramp is 2 meters from the bed, and now it's that the ramp is on the bed. This works.

**Second mutation**: changed from landing the ball in/on the hexagonal bin, to landing the dodgeball on the golfball, which is rather difficult (if not impossible), but still valid

In [108]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[0]], 4)

With random seed 4, mutated:



**Original note**: Changed the setup slightly -- before it was `(game-conserved (= (distance ?b ?r) 2))`, that is, the ramp is 2 meters from the bed, and now it's that the ramp is on the bed. This works.

**Second mutation**: changed from landing on the in/on the hexagonal bin, to catching it and holding it again. Totally valid.

In [110]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[0]], 6)

With random seed 6, mutated:



**Original note:** I think the other substitution method also did something like this -- though it bounced it off the curved ramp, and this bounces it off the hexagonal bin

**Second mutation:** this changes the objects the game is played with again, from the dodgeball and bin to the basketball and bin

In [115]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[3]], 1)

With random seed 1, mutated:



**Original note:** I think the other substitution method also did something like this -- though it bounced it off the curved ramp, and this bounces it off the hexagonal bin

**Second mutation:** Another minor change that seems to work -- instead of measuring the distance to the agent when the ball is caught, measure the distance to the wall

In [120]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[3]], 6)

With random seed 6, mutated:



**Original note:** I think the other substitution method also did something like this -- though it bounced it off the curved ramp, and this bounces it off the hexagonal bin

**Second mutation:** Changed from bouncing the ball off the wall to landing it on the bin.

In [121]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[3]], 7)

With random seed 8, mutated:



**Original note:** Throw the cellphone such that it is adjacent to the doggie bed, rather than such that it lands on it. Cute and valid.

**Second mutation:** Added the constraint that the cellphone must be thrown with the agent's eyes closed.

In [125]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[4]], 1)

With random seed 1, mutated:



**Original comment:** I'm mildly stunned this worked, but it worked -- this is what was there before: `(exists (?b2 - (either cube_block tall_cylindrical_block short_cylindrical_block)) (on ?b1 ?b2))`. Now it asks if there's another game object, that is in the building, and `?b1` is on the game object `?o2` -- it basically changes the game from "build a tower with the blocks" to "build a tower, starting from a block, and using any object you want, but you only get points for each block you use" -- so you could use other objects to stabilize, for example.

**Second mutation:** This time it changes the rules of consturction again, starting from a block (and counting at most one block), but then scoring for every ball (dodgeball or beachball) in the structure.

In [136]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[6]], 0)

With random seed 0, mutated:



**Original comment:** I'm mildly stunned this worked, but it worked -- this is what was there before: `(exists (?b2 - (either cube_block tall_cylindrical_block short_cylindrical_block)) (on ?b1 ?b2))`. Now it asks if there's another game object, that is in the building, and `?b1` is on the game object `?o2` -- it basically changes the game from "build a tower with the blocks" to "build a tower, starting from a block, and using any object you want, but you only get points for each block you use" -- so you could use other objects to stabilize, for example.

**Second mutation:** Another change to the construction rules, this time to start building from a ball touching the floor.

In [138]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[6]], 2)

With random seed 3, mutated:



**Original comment:** I'm mildly stunned this worked, but it worked -- this is what was there before: `(exists (?b2 - (either cube_block tall_cylindrical_block short_cylindrical_block)) (on ?b1 ?b2))`. Now it asks if there's another game object, that is in the building, and `?b1` is on the game object `?o2` -- it basically changes the game from "build a tower with the blocks" to "build a tower, starting from a block, and using any object you want, but you only get points for each block you use" -- so you could use other objects to stabilize, for example.

**Second mutation:** This time the initial block (which is the base of the construction, theoretically) must be adjacent to the agent and touched by the agent when the game ends. 

In [140]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[6]], 4)

With random seed 5, mutated:



In [147]:
visualize(second_step_mutations_dict[INDICES_TO_MUTATE[4]], 2)

With random seed 2, mutated:



# Debugging


In [123]:
find_mutations(single_step_mutated_asts[44])

{'variables': [{'var_names': ['?r'], 'var_type': 'large_triangular_ramp', 'parseinfo': [None, 'variable_type_def', 137, 163, 0, 0]}], 'parseinfo': [None, 'variable_list', 135, 164, 0, 0], 'mutation': 0}


[{'variables': [{'var_names': ['?r'], 'var_type': 'large_triangular_ramp', 'parseinfo': [None, 'variable_type_def', 137, 163, 0, 0]}], 'parseinfo': [None, 'variable_list', 135, 164, 0, 0], 'mutation': 0}]

In [32]:
new_ast_to_buffer(single_step_mutated_asts, 44)
[line for line in ast_printer.BUFFER if 'color' in line]

['<div style="margin-left: 60px"><span style="">(exists</span> <span style="color: #1f77b4">(<span style="">?r - large_triangular_ramp</span>)</span> </div>']

In [33]:
for idx, ast in enumerate(single_step_mutated_asts):
    new_ast_to_buffer(single_step_mutated_asts, idx)
    if sum(['color' in line for line in ast_printer.BUFFER]) == 0:
        print(idx)

In [ ]:
random.seed(0)
src_ast = single_step_mutated_asts[INDICES_TO_MUTATE[0]][0]
src_ast_copy = copy_ast(grammar_parser, src_ast)
copy_mutation_tags(src_ast, src_ast_copy)